In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

import os

import cv2

import scipy
from scipy import signal

import h5py

import time
import datetime as dt
from tqdm import tqdm


import iminuit
from iminuit import Minuit, describe

from pprint import pprint # we use this to pretty print some stuff later

import glob
import sys

sys.path.append('/home/analysis_user/New_trap_code/Tools/')

import BeadDataFile
from discharge_tools import *
from AnaUtil import *


from joblib import Parallel, delayed
import multiprocessing


In [ ]:
### load images in .bmp format
def load_img_files(path,print_list=True):
    # read in the data 
    files = glob.glob(path)
    img_files = []
    for i in range(0,len(files)):
        img_files.append(cv2.imread(files[i],0))
        if(print_list==True):print(files[i]) 
    return img_files

def load_npy_files(path,print_list=True):
    files = glob.glob(path)
    img_files = []
    for i in range(0,len(files)):
        img_files.append(np.load(files[i],0))
        if(print_list==True):print(files[i]) 
    return img_files    

In [ ]:
def laplacian(img):
    lap = np.sum(cv2.Laplacian(img,cv2.CV_64F)*cv2.Laplacian(img,cv2.CV_64F))
    return lap

In [5]:
def gaussian_attr_height_fit(img,low_x_lim=100,up_x_lim=1100,low_y_lim=400,up_y_lim=500,upper_area=300,up_lim_width=15):

    def chisquare_1d(function, functionparams, data_x, data_y,data_y_error):
        chisquarevalue=np.sum(np.power(np.divide(np.subtract(function(data_x,functionparams),data_y),data_y_error),2))
        ndf = len(data_y)-len(functionparams)
        #print(ndf)
        return (chisquarevalue, ndf)
    
    def chisquare_gaussian(area,mean,sigma,constant1,constant2):
        return chisquare_1d(function=gaussian,functionparams=[area,mean,sigma,constant1,constant2],data_x=data_x,data_y=data_y,data_y_error=data_y_error)[0]
   

    area=0
    mean=0
    sigma=0
    constant1=0
    constant2=0
    low_fitrange=50
    up_fitrange=10


    img2 = img.transpose()
    data_y_input = np.mean(img2[low_x_lim:up_x_lim],axis=0) # give y data 
    data_x = np.arange(np.argmax(data_y_input)-low_fitrange,np.argmax(data_y_input)+up_fitrange,1)#range(1024) # give x data
    data_y = np.mean(img2[low_x_lim:up_x_lim],axis=0)[np.argmax(data_y_input)-low_fitrange:np.argmax(data_y_input)+up_fitrange] # give y data 
    data_y_error = np.sqrt(data_y) # give y uncertainty
    low_lim_mean = low_y_lim
    up_lim_mean = up_y_lim

    m=Minuit(chisquare_gaussian, 
             area = 100, # set start parameter
             error_area = 1,
             limit_area= (0,upper_area), # if you want to limit things
             #fix_area = "True", # you can also fix it
             mean = np.argmax(np.mean(img2[low_x_lim:up_x_lim],axis=0)),
             error_mean = 1,
             #fix_mean = "False",
             #limit_mean = (450,490),
             sigma = 4.5,
             error_sigma = 1,
             limit_sigma=(0,up_lim_width),
             constant1 = 0,
             fix_constant1=True,
             error_constant1 = 1,
             constant2 = 55,
             error_constant2 = 1,
             fix_constant2=False,
             errordef = 1,
             print_level=0)
    #print('Now proceed with the fit.')
    m.migrad(ncall=500000)
    #m.minos(), if you need fancy mapping
    chisquare=m.fval
    return m.values['mean'],m